In [1]:
import sqlalchemy

verifico version

In [2]:
sqlalchemy.__version__

'1.2.8'

definir el nombre de la base de datos

In [3]:
DATABASE_FILE = "database.db"

remover db existente, para eso crearemos un metodo helper para eso

In [4]:
import os

def delete_database_file():
    if os.path.isfile(DATABASE_FILE):
        os.remove(DATABASE_FILE)

In [5]:
delete_database_file()

crear un objeto engine que usara sqlite

In [6]:
engine = sqlalchemy.create_engine('sqlite:///database.db', echo=True)

In [7]:
metadata = sqlalchemy.MetaData()

users = sqlalchemy.Table(
    'users',
    metadata,
    sqlalchemy.Column('id', sqlalchemy.Integer, primary_key=True),
    sqlalchemy.Column('name', sqlalchemy.String),
    sqlalchemy.Column('fullname', sqlalchemy.String),
)

addresses = sqlalchemy.Table(
    'addresses',
    metadata,
    sqlalchemy.Column('id', sqlalchemy.Integer, primary_key=True),
    sqlalchemy.Column('user_id', None, sqlalchemy.ForeignKey('users.id')),
    sqlalchemy.Column('email_address', sqlalchemy.String, nullable=False)
)

crear las tablas en la base de datos pero en serio y usando SQL real

In [8]:
delete_database_file()
metadata.create_all(engine)

2018-06-20 05:17:23,127 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-06-20 05:17:23,130 INFO sqlalchemy.engine.base.Engine ()
2018-06-20 05:17:23,132 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-06-20 05:17:23,134 INFO sqlalchemy.engine.base.Engine ()
2018-06-20 05:17:23,147 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2018-06-20 05:17:23,148 INFO sqlalchemy.engine.base.Engine ()
2018-06-20 05:17:23,153 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("addresses")
2018-06-20 05:17:23,156 INFO sqlalchemy.engine.base.Engine ()
2018-06-20 05:17:23,160 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2018-06-20 05:17:23,162 INFO sqlalchemy.engine.base.Engine ()
2018-06-20 05:17:23,166 INFO sqlalchemy.engine.base.Engine COMMIT
2018-06-20 05:17:23,169 INFO sqlalchemy.engine.b

# Insert Operations

In [9]:
#se extrae de la tabla misma los objetos insert
ins = users.insert()
ins

In [10]:
str(ins)

'INSERT INTO users (id, name, fullname) VALUES (:id, :name, :fullname)'

In [11]:
ins = ins.values(name='jack', fullname='Jack Jones')
str(ins)

'INSERT INTO users (name, fullname) VALUES (:name, :fullname)'

In [12]:
ins.compile().params

{'fullname': 'Jack Jones', 'name': 'jack'}

# Execution

In [13]:
conn = engine.connect()
conn

In [14]:
result = conn.execute(ins)

2018-06-20 05:17:23,315 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname) VALUES (?, ?)
2018-06-20 05:17:23,319 INFO sqlalchemy.engine.base.Engine ('jack', 'Jack Jones')
2018-06-20 05:17:23,322 INFO sqlalchemy.engine.base.Engine COMMIT


In [15]:
result.inserted_primary_key

[1]

In [16]:
#add another one
conn.execute(ins, id=2, name='wendy', fullname='Wendy Williams')

2018-06-20 05:17:23,366 INFO sqlalchemy.engine.base.Engine INSERT INTO users (id, name, fullname) VALUES (?, ?, ?)
2018-06-20 05:17:23,368 INFO sqlalchemy.engine.base.Engine (2, 'wendy', 'Wendy Williams')
2018-06-20 05:17:23,371 INFO sqlalchemy.engine.base.Engine COMMIT


insertamos varios a la vez

In [17]:
conn.execute(addresses.insert(), [
    {'user_id': 1, 'email_address' : 'jack@yahoo.com'},
    {'user_id': 1, 'email_address' : 'jack@msn.com'},
    {'user_id': 2, 'email_address' : 'www@www.org'},
    {'user_id': 2, 'email_address' : 'wendy@aol.com'},
])

2018-06-20 05:17:23,398 INFO sqlalchemy.engine.base.Engine INSERT INTO addresses (user_id, email_address) VALUES (?, ?)
2018-06-20 05:17:23,400 INFO sqlalchemy.engine.base.Engine ((1, 'jack@yahoo.com'), (1, 'jack@msn.com'), (2, 'www@www.org'), (2, 'wendy@aol.com'))
2018-06-20 05:17:23,403 INFO sqlalchemy.engine.base.Engine COMMIT


# Select

In [18]:
s = sqlalchemy.sql.select([users])
result = conn.execute(s)

2018-06-20 05:17:23,424 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users
2018-06-20 05:17:23,426 INFO sqlalchemy.engine.base.Engine ()


In [19]:
#Iterate over the result object
for row in result:
    print(row)

(1, u'jack', u'Jack Jones')
(2, u'wendy', u'Wendy Williams')


In [34]:
result = conn.execute(s)
row = result.fetchone()
print(row.name)
row = result.fetchone()
print(row.name)

2018-06-20 05:22:45,023 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users
2018-06-20 05:22:45,025 INFO sqlalchemy.engine.base.Engine ()
jack
wendy


In [35]:
result.close()

# More detailed selects

In [38]:
users

Table('users', MetaData(bind=None), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('name', String(), table=<users>), Column('fullname', String(), table=<users>), schema=None)

In [39]:
users.c

In [40]:
users.c.name

Column('name', String(), table=<users>)

In [41]:
users.c.fullname

Column('fullname', String(), table=<users>)

In [43]:
s = sqlalchemy.sql.select([users])
str(s)

'SELECT users.id, users.name, users.fullname \nFROM users'

In [49]:
s = sqlalchemy.sql.select([users.c.name, addresses.c.email_address])
str(s)

'SELECT users.name, addresses.email_address \nFROM users, addresses'

In [50]:
result = conn.execute(s)

2018-06-20 05:30:59,297 INFO sqlalchemy.engine.base.Engine SELECT users.name, addresses.email_address 
FROM users, addresses
2018-06-20 05:30:59,299 INFO sqlalchemy.engine.base.Engine ()


In [52]:
for row in result:
    print(row)

(u'jack', u'jack@yahoo.com')
(u'jack', u'jack@msn.com')
(u'jack', u'www@www.org')
(u'jack', u'wendy@aol.com')
(u'wendy', u'jack@yahoo.com')
(u'wendy', u'jack@msn.com')
(u'wendy', u'www@www.org')
(u'wendy', u'wendy@aol.com')


## Where clause

In [63]:
s = sqlalchemy.sql.select([users.c.name, addresses.c.email_address])
s = s.where(users.c.id == addresses.c.user_id)

In [64]:
result = conn.execute(s)
for row in result:
    print(row)

2018-06-20 05:41:42,162 INFO sqlalchemy.engine.base.Engine SELECT users.name, addresses.email_address 
FROM users, addresses 
WHERE users.id = addresses.user_id
2018-06-20 05:41:42,164 INFO sqlalchemy.engine.base.Engine ()
(u'jack', u'jack@yahoo.com')
(u'jack', u'jack@msn.com')
(u'wendy', u'www@www.org')
(u'wendy', u'wendy@aol.com')
